In [1]:
import os
import re
import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

# SERVICE_TYPE = 'tos_cross_sell'
# DATASET_ID = '{}_dataset'.format(SERVICE_TYPE)
# PROJECT_ID = mapping['PROJECT_ID']
# RESOURCE_BUCKET = mapping['resources_bucket']
# FILE_BUCKET = mapping['gcs_csv_bucket']
# REGION = mapping['REGION']
# MODEL_ID = '5060'
# FOLDER_NAME = 'xgb_{}_{}_predict_deploy'.format(SERVICE_TYPE, MODEL_ID)
# QUERIES_PATH = 'vertex_pipelines/' + FOLDER_NAME + '/queries/'

SERVICE_TYPE = 'tos_cross_sell'
DATASET_ID = 'tos_cross_sell'
PROJECT_ID = 'divg-josh-pr-d1cc3a' #mapping['PROJECT_ID']
RESOURCE_BUCKET = 'divg-josh-pr-d1cc3a-default' #mapping['resources_bucket']
FILE_BUCKET = 'divg-josh-pr-d1cc3a-default' #mapping['gcs_csv_bucket']
REGION = 'northamerica-northeast1' #mapping['REGION']
MODEL_ID = '5060'
FOLDER_NAME = 'xgb_tos_cross_sell_train_deploy'.format(MODEL_ID)
QUERIES_PATH = 'vertex_pipelines/' + FOLDER_NAME + '/queries/'

scoringDate = date.today() - relativedelta(days=5)

# current day views
CONSL_VIEW_NAME = '{}_pipeline_consl_data_curr_bi_layer'.format(SERVICE_TYPE)  # done
FFH_BILLING_VIEW_NAME = '{}_pipeline_ffh_billing_data_curr_bi_layer'.format(SERVICE_TYPE)  # done
HS_USAGE_VIEW_NAME = '{}_pipeline_hs_usage_data_curr_bi_layer'.format(SERVICE_TYPE)  # done
DEMO_INCOME_VIEW_NAME = '{}_pipeline_demo_income_data_curr_bi_layer'.format(SERVICE_TYPE)  # done
PROMO_EXPIRY_VIEW_NAME = '{}_pipeline_promo_expiry_data_curr_bi_layer'.format(SERVICE_TYPE)  # done
GPON_COPPER_VIEW_NAME = '{}_pipeline_gpon_copper_data_curr_bi_layer'.format(SERVICE_TYPE)  # done
CLCKSTRM_TELUS_VIEW_NAME = '{}_pipeline_clckstrm_telus_curr_bi_layer'.format(SERVICE_TYPE)
ALARMDOTCOM_APP_USAGE_VIEW_NAME = '{}_pipeline_alarmdotcom_app_usage_curr_bi_layer'.format(SERVICE_TYPE)
TOS_ACTIVE_BANS_VIEW_NAME = '{}_pipeline_tos_active_bans_curr_bi_layer'.format(SERVICE_TYPE) 

# dates
SCORE_DATE = scoringDate.strftime('%Y%m%d')  # date.today().strftime('%Y%m%d')
SCORE_DATE_DASH = scoringDate.strftime('%Y-%m-%d')
SCORE_DATE_MINUS_6_MOS_DASH = ((scoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_START_DASH = (scoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_END_DASH = ((scoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_YEAR = ((scoringDate.replace(day=1)) - timedelta(days=1)).year
SCORE_DATE_LAST_MONTH_MONTH = ((scoringDate.replace(day=1)) - timedelta(days=1)).month

SCORE_DATE_DELTA = 0
SCORE_DATE_VAL_DELTA = 0
TICKET_DATE_WINDOW = 30  # Days of ticket data to be queried

ACCOUNT_CONSL_QUERY_PATH = QUERIES_PATH + 'create_input_account_consl_query.txt'
ACCOUNT_GPON_COPPER_QUERY_PATH = QUERIES_PATH + 'create_input_account_gpon_copper_query.txt'
ACCOUNT_PROMO_EXPIRY_QUERY_PATH = QUERIES_PATH + 'create_input_account_promo_expiry_query.txt'
ACCOUNT_DEMO_INCOME_QUERY_PATH = QUERIES_PATH + 'create_input_account_demo_income_query.txt'
ACCOUNT_HS_USAGE_QUERY_PATH = QUERIES_PATH + 'create_input_account_hs_usage_query.txt'
ACCOUNT_FFH_BILLING_QUERY_PATH = QUERIES_PATH + 'create_input_account_ffh_billing_query.txt'
ACCOUNT_CLCKSTRM_TELUS_QUERY_PATH = QUERIES_PATH + 'create_input_account_clckstrm_telus_query.txt'
ACCOUNT_ALARMDOTCOM_APP_USAGE_QUERY_PATH = QUERIES_PATH + 'create_input_account_alarmdotcom_app_usage_query.txt'
ACCOUNT_TOS_ACTIVE_BANS_QUERY_PATH = QUERIES_PATH + 'create_input_account_tos_active_bans_query.txt'


In [2]:
def batch_prediction(
        project_id: str,
        dataset_id: str,
        file_bucket: str,
        service_type: str,
        score_table: str,
        score_date_dash: str
        # metrics: Output[Metrics],
        # metricsc: Output[ClassificationMetrics],
):
    import time
    import pandas as pd
    import numpy as np
    import pickle
    from datetime import date
    from dateutil.relativedelta import relativedelta
    from google.cloud import bigquery
    from google.cloud import storage

    MODEL_ID = '5060'

    def if_tbl_exists(bq_client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            bq_client.get_table(table_ref)
            return True
        except NotFound:
            return False


        
    def upsert_table(project_id, dataset_id, table_id, sql, result):
        new_values = ',\n'.join(result.apply(lambda row: row_format(row), axis=1))
        new_sql = sql.format(proj_id=project_id, dataset_id=dataset_id, table_id=table_id,
                             new_values=new_values)
        
        def get_gcp_bqclient(project_id, use_local_credential=True):
            token = os.popen('gcloud auth print-access-token').read()
            token = re.sub(f'\n$', '', token)
            credentials = google.oauth2.credentials.Credentials(token)

            bq_client = bigquery.Client(project=project_id)
            if use_local_credential:
                bq_client = bigquery.Client(project=project_id, credentials=credentials)
            return bq_client

        # bq_client = bigquery.Client(project=project_id)
        bq_client = get_gcp_bqclient(project_id)
        
        code = bq_client.query(new_sql)
        time.sleep(5)

    def row_format(row):
        values = row.values
        new_values = ""
        v = str(values[0]) if not pd.isnull(values[0]) else 'NULL'
        if 'str' in str(type(values[0])):
            new_values += f"'{v}'"
        else:
            new_values += f"{v}"

        for i in range(1, len(values)):
            v = str(values[i]) if not pd.isnull(values[i]) else 'NULL'
            if 'str' in str(type(values[i])):
                new_values += f",'{v}'"
            else:
                new_values += f",{v}"
        return '(' + new_values + ')'

    def generate_sql_file(ll):
        s = 'MERGE INTO `{proj_id}.{dataset_id}.{table_id}` a'
        s += " USING UNNEST("
        s += "[struct<"
        for i in range(len(ll) - 1):
            v = ll[i]
            s += "{} {},".format(v[0], v[1])
        s += "{} {}".format(ll[-1][0], ll[-1][1])
        s += ">{new_values}]"
        s += ") b"
        s += " ON a.ban = b.ban and a.score_date = b.score_date"
        s += " WHEN MATCHED THEN"
        s += " UPDATE SET "
        s += "a.{}=b.{},".format(ll[0][0], ll[0][0])
        for i in range(1, len(ll) - 1):
            v = ll[i]
            s += "a.{}=b.{},".format(v[0], v[0])
        s += "a.{}=b.{}".format(ll[-1][0], ll[-1][0])
        s += " WHEN NOT MATCHED THEN"
        s += " INSERT("
        for i in range(len(ll) - 1):
            v = ll[i]
            s += "{},".format(v[0])
        s += "{})".format(ll[-1][0])
        s += " VALUES("
        for i in range(len(ll) - 1):
            s += "b.{},".format(ll[i][0])
        s += "b.{}".format(ll[-1][0])
        s += ")"

        return s

    MODEL_PATH = '{}_xgb_models/'.format(service_type)
    df_score = pd.read_csv('gs://{}/{}_score.csv.gz'.format(file_bucket, service_type), compression='gzip')
    df_score.dropna(subset=['ban'], inplace=True)
    df_score.reset_index(drop=True, inplace=True)
    print('......scoring data loaded:{}'.format(df_score.shape))
    time.sleep(10)

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(file_bucket)
    blobs = storage_client.list_blobs(file_bucket, prefix='{}{}_models_xgb_'.format(MODEL_PATH, service_type))

    model_lists = []
    for blob in blobs:
        model_lists.append(blob.name)

    blob = bucket.blob(model_lists[-1])
    blob_in = blob.download_as_string()
    model_dict = pickle.loads(blob_in)
    model_xgb = model_dict['model']
    features = model_dict['features']
    print('...... model loaded')
    time.sleep(10)

    ll = [('ban', 'string'), ('score_date', 'string'), ('model_id', 'string'), ('score', 'float64')]
    sql = generate_sql_file(ll)

    df_score['ban'] = df_score['ban'].astype(int)
    print('.... scoring for {} tos cross sell bans base'.format(len(df_score)))

    # get full score to cave into bucket
    pred_prob = model_xgb.predict_proba(df_score[features], ntree_limit=model_xgb.best_iteration)[:, 1]
    result = pd.DataFrame(columns=['ban', 'score_date', 'model_id', 'score'])
    result['score'] = list(pred_prob)
    result['score'] = result['score'].fillna(0.0).astype('float64')
    result['ban'] = list(df_score['ban'])
    result['ban'] = result['ban'].astype('str')
    result['score_date'] = score_date_dash
    result['model_id'] = MODEL_ID

    result.to_csv('gs://{}/ucar/{}_prediction.csv.gz'.format(file_bucket, service_type), compression='gzip',
                  index=False)
    time.sleep(60)

    batch_size = 1000
    n_batchs = int(df_score.shape[0] / batch_size) + 1
    print('...... will upsert {} batches'.format(n_batchs))

    # start batch prediction
    all_scores = np.array(result['score'].values)
    for i in range(n_batchs):

        s, e = i * batch_size, (i + 1) * batch_size
        if e >= df_score.shape[0]:
            e = df_score.shape[0]

        df_temp = df_score.iloc[s:e]
        pred_prob = all_scores[s:e]
        batch_result = pd.DataFrame(columns=['ban', 'score_date', 'model_id', 'score'])
        batch_result['score'] = list(pred_prob)
        batch_result['score'] = batch_result['score'].fillna(0.0).astype('float64')
        batch_result['ban'] = list(df_temp['ban'])
        batch_result['ban'] = batch_result['ban'].astype('str')
        batch_result['score_date'] = score_date_dash
        batch_result['model_id'] = MODEL_ID

        upsert_table(project_id,
                     dataset_id,
                     score_table,
                     sql,
                     batch_result,
                     )
        if i % 20 == 0:
            print('predict for batch {} done'.format(i), end=' \n')

    time.sleep(120)


    #-------------------------------------------------------complete upto here----------------------------------------------------------------#


def postprocess(
        project_id: str,
        file_bucket: str,
        service_type: str,
        score_date_dash: str,
):
    import time
    import pandas as pd
    from google.cloud import bigquery

    MODEL_ID = '5060'
    file_name = 'gs://{}/ucar/{}_prediction.csv.gz'.format(file_bucket, service_type)
    df_orig = pd.read_csv(file_name, compression='gzip')
    df_orig.dropna(subset=['ban'], inplace=True)
    df_orig.reset_index(drop=True, inplace=True)
    df_orig['scoring_date'] = score_date_dash
    df_orig.ban = df_orig.ban.astype(int)
    df_orig = df_orig.rename(columns={'ban': 'bus_bacct_num', 'score': 'score_num'})
    df_orig.score_num = df_orig.score_num.astype(float)
    df_orig['decile_grp_num'] = pd.qcut(df_orig['score_num'], q=10, labels=False)
    df_orig.decile_grp_num = df_orig.decile_grp_num + 1
    df_orig['percentile_pct'] = df_orig.score_num.rank(pct=True)
    df_orig['predict_model_nm'] = 'FFH TOS CROSS SELL Model - DIVG'
    df_orig['model_type_cd'] = 'FFH'
    df_orig['subscriber_no'] = ""
    df_orig['prod_instnc_resrc_str'] = ""
    df_orig['service_instnc_id'] = ""
    df_orig['segment_nm'] = ""
    df_orig['segment_id'] = ""
    df_orig['classn_nm'] = ""
    df_orig['predict_model_id'] = MODEL_ID
    df_orig.drop(columns=['model_id', 'score_date'], axis=1, inplace=True)

    get_cust_id = """
    WITH bq_snpsht_max_date AS(
    SELECT PARSE_DATE('%Y%m%d', MAX(partition_id)) AS max_date
        FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.INFORMATION_SCHEMA.PARTITIONS` 
    WHERE table_name = 'bq_prod_instnc_snpsht' 
        AND partition_id <> '__NULL__'
    ),
    -- BANs can have multiple Cust ID. Create rank by product type and status, prioritizing ban/cust id with active FFH products
    rank_prod_type AS (
    SELECT DISTINCT
        bacct_bus_bacct_num,
        consldt_cust_bus_cust_id AS cust_id,
        CASE WHEN pi_prod_instnc_resrc_typ_cd IN ('SING', 'HSIC', 'TTV', 'SMHM', 'STV', 'DIIC') AND pi_prod_instnc_stat_cd = 'A' THEN 1
                WHEN pi_prod_instnc_resrc_typ_cd IN ('SING', 'HSIC', 'TTV', 'SMHM', 'STV', 'DIIC') THEN 2
                WHEN pi_prod_instnc_stat_cd = 'A' THEN 3
                ELSE 4
                END AS prod_rank
    FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht`
    CROSS JOIN bq_snpsht_max_date
    WHERE CAST(prod_instnc_ts AS DATE)=bq_snpsht_max_date.max_date
    AND bus_prod_instnc_src_id = 1001
    ),
    --Rank Cust ID
    rank_cust_id AS (
    SELECT DISTINCT
        bacct_bus_bacct_num,
        cust_id,
        RANK() OVER(PARTITION BY bacct_bus_bacct_num
                        ORDER BY prod_rank,
                                    cust_id) AS cust_id_rank               
    FROM rank_prod_type
    )
    --Select best cust id
    SELECT bacct_bus_bacct_num,
        cust_id
    FROM rank_cust_id
    WHERE cust_id_rank = 1
    """

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    df_cust = bq_client.query(get_cust_id).to_dataframe()
    df_final = df_orig.set_index('bus_bacct_num').join(df_cust.set_index('bacct_bus_bacct_num')).reset_index()
    df_final = df_final.rename(columns={'index': 'bus_bacct_num', 'cust_bus_cust_id': 'cust_id'})
    df_final = df_final.sort_values(by=['score_num'], ascending=False)
    df_final.to_csv(file_name, compression='gzip', index=False)
    time.sleep(300)

def pipeline(
            project_id: str = PROJECT_ID,
            region: str = REGION,
            resource_bucket: str = RESOURCE_BUCKET,
            file_bucket: str = FILE_BUCKET
    ):
    batch_prediction_op = batch_prediction(
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
        file_bucket=FILE_BUCKET,
        service_type=SERVICE_TYPE,
        score_date_dash=SCORE_DATE_DASH,
        score_table='bq_tos_cross_sell_score',
    )
    postprocessing_op = postprocess(
        project_id=PROJECT_ID,
        file_bucket=FILE_BUCKET,
        service_type=SERVICE_TYPE,
        score_date_dash=SCORE_DATE_DASH,
    )
    batch_prediction_op
      postprocessing_op

    return pipeline

In [3]:
pipeline(project_id = PROJECT_ID, region = REGION, resource_bucket = RESOURCE_BUCKET, file_bucket = FILE_BUCKET)

......scoring data loaded:(2077417, 69)
...... model loaded
.... scoring for 2077417 tos cross sell bans base


/opt/conda/lib/python3.8/site-packages/xgboost/core.py:90: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


...... will upsert 2078 batches
predict for batch 0 done 
predict for batch 20 done 
predict for batch 40 done 
predict for batch 60 done 
predict for batch 80 done 
predict for batch 100 done 
predict for batch 120 done 
predict for batch 140 done 
predict for batch 160 done 
predict for batch 180 done 
predict for batch 200 done 
predict for batch 220 done 
predict for batch 240 done 
predict for batch 260 done 
predict for batch 280 done 
predict for batch 300 done 
predict for batch 320 done 
predict for batch 340 done 
predict for batch 360 done 
predict for batch 380 done 
predict for batch 400 done 
predict for batch 420 done 
predict for batch 440 done 
predict for batch 460 done 
predict for batch 480 done 
predict for batch 500 done 
predict for batch 520 done 
predict for batch 540 done 
predict for batch 560 done 
predict for batch 580 done 
predict for batch 600 done 
predict for batch 620 done 
predict for batch 640 done 
predict for batch 660 done 
predict for batch 680 

<function __main__.pipeline(project_id: str = 'divg-josh-pr-d1cc3a', region: str = 'northamerica-northeast1', resource_bucket: str = 'divg-josh-pr-d1cc3a-default', file_bucket: str = 'divg-josh-pr-d1cc3a-default')>